In [0]:
%pip install -U -qqqq mlflow unitycatalog-ai[databricks] databricks-langchain

**重要** このノートブックで開発やテストを行う際には、次のセルのコードが必要です。  
ドライバーノートブックからこのノートブックを実行する場合は、次のセルの行をコメントアウトする必要があります。

In [0]:
# dbutils.library.restartPython()

In [0]:
# 必要なライブラリのインポート
import os
import mlflow
import json

from databricks_langchain import DatabricksVectorSearch
from langchain_community.chat_models.databricks import ChatDatabricks
from langchain_core.prompts import PromptTemplate
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

### MLflow Tracingを有効にする
以下のような場合は、MLflow Tracingを有効にします。
- このノートブックでトレースの視覚化を表示する
- 推論テーブルを使用して本番環境でトレースをキャプチャする
- アプリケーションを評価する

In [0]:
mlflow.langchain.autolog()

### 1. Databricks Vector Search を利用した RAG の retrieve 
DatabricksVectorSearch のインスタンスを作成  
[参考URL](https://python.langchain.com/docs/integrations/vectorstores/databricks_vector_search/)


In [0]:
db_vector_search = DatabricksVectorSearch(
    endpoint="vs_endpoint",
    index_name="trainer_catalog.05_vector_search_index_for_nssol.product_documentation_vs_index",
)

def retrieve_manual_sections(query: str) -> str:
    """
    ユーザーの問い合わせをもとに、Vector Search マニュアルの関連セクションを取得します。
    """
    docs = db_vector_search.similarity_search(query, k=3)
    sections = "\n".join([doc.page_content for doc in docs])
    return sections if sections else "適切なセクションが見つかりませんでした。"

### 2. Unity Catalog の関数を利用して製品情報を取得

ここでは、事前に Unity Catalog に登録済みの関数を利用する定義を行います。  
"openhack2025.04_ai_agent_for_demo.product_with_many_inquiries" 関る鵜を利用します。


In [0]:
# catalog = "openhack2025"
# db_name = "04_ai_agent_for_demo"
# warehouse_id = "12a4cd99e4b901b6"
func_name = "trainer_catalog.03_data_analysis_by_gen_ai_for_nssol.product_with_many_inquiries"

AI Functionを作る処理を説明。

In [0]:
# Databricks Function クライアントの作成
client = DatabricksFunctionClient()

# UCFunctionToolkit を利用して、Unity Catalog 関数をツールとして登録
toolkit = UCFunctionToolkit(
    function_names=[func_name],
    client=client
)

# 登録されたツールから対象の関数を取得
product_info_function = toolkit.tools[0]

def get_product_info(query: str) -> str:
    """
    ユーザーの問い合わせを引数として Unity Catalog に登録された関数を呼び出し、
    問い合わせの多い製品情報を取得します。
    """
    try:
        result = product_info_function.run(query)
        return result
    except Exception as e:
        return f"Error calling product info function: {str(e)}"

### 3. LLM を利用した回答生成（ChatDatabricks の利用）

In [0]:
# LLMの初期化
llm = ChatDatabricks(
    endpoint="openhack-gpt-4o",  # エンドポイントは環境に合わせて設定
    temperature=0.0,
    streaming=True
)

In [0]:
# プロンプトテンプレート：ユーザーの問い合わせ、製品情報、マニュアルセクションを統合
prompt = PromptTemplate(
    template="""
あなたは取扱製品の問い合わせ対応ナレッジを持つ、高度なアシスタントです。 問い合わせに対する調査や対応方針の提案を行います。
回答にあたっては、以下のツールを適切に使ってください：

User Query: {user_query}

Product Information (from Unity Catalog function):
{product_info}

Relevant Manual Sections (from Vector Search):
{manual_sections}

分からない場合は分からない旨を回答してください。
また、日本語で回答してください。
""",
    input_variables=["user_query", "product_info", "manual_sections"]
)


In [0]:
def rag_agent(user_query: str) -> str:
    """
    RAG パイプライン：Vector Search と Unity Catalog 関数を利用して情報を取得し、LLM による回答を生成します。
    """
    # 1. 修理マニュアルの関連セクションを取得
    manual_sections = retrieve_manual_sections(user_query)
    
    # 2. 問い合わせの多い製品情報を取得
    product_info = get_product_info(user_query)
    
    # 3. プロンプトに情報を統合
    full_prompt = prompt.format(
        user_query=user_query,
        product_info=product_info,
        manual_sections=manual_sections
    )
    # 4. LLM にプロンプトを渡して回答を生成
    response = llm.invoke(full_prompt)

    # response がオブジェクトの場合、content 属性を返す
    if hasattr(response, "content"):
        return response.content
    
    # もしくは辞書の場合
    if isinstance(response, dict) and "content" in response:
        return response["content"]
    # それ以外はそのまま返す
    return response

In [0]:
def extract_latest_user_message(messages):
    """
    会話履歴から、ユーザーの最新の発話を抽出します。
    """
    # 逆順に検索して、最初に見つかったユーザーメッセージを返す
    for msg in reversed(messages):
        if msg.get("role") == "user":
            return msg.get("content", "")
    return ""

def conversation_agent(messages):
    """
    会話履歴（メッセージのリスト）を入力として受け取り、最新のユーザー発話を使って
    rag_agent（先ほど定義したRAGパイプライン）を呼び出します。
    """
    user_query = extract_latest_user_message(messages)
    return rag_agent(user_query)

In [0]:
# ===============================================
# 4. 実行例（デモ）
# ===============================================
if __name__ == "__main__":
    # 会話履歴の例（システムメッセージやユーザーメッセージを含む）
    conversation = [
        {"role": "system", "content": "あなたはインテリジェントなアシスタントです。"},
        {"role": "user", "content": "問い合わせの多い製品の修理手順を教えてください。"}
    ]
    # user_query = "問い合わせの多い製品の修理手順を教えてください。"
    # answer = rag_agent(user_query)
    answer = conversation_agent(conversation)

    # シンプルなテキストとして出力
    print("AI Agent Answer (Text):")
    print(answer)
    
    # # JSON 形式として出力する場合
    # json_output = json.dumps({"answer": answer}, ensure_ascii=False, indent=2)
    # print("\nAI Agent Answer (JSON):")
    # print(json_output)
    
    # オプション：MLflow によるトレーシング（必要に応じて）
    

In [0]:
mlflow.models.set_model(model=llm)